In [26]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

Load the pre-trained FLAN-T5-base model and its tokenizer directly from HuggingFace. Notice that we will be using the small version of FLAN-T5. Setting torch_dtype=torch.bfloat16 specifies the memory type to be used by this model.

In [27]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Experimenting with the DialogSum Hugging Face dataset. It contains 10,000+ dialogues with the corresponding manually labeled summaries and topics.

In [28]:
#dataset
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

You need to convert the dialog-summary (prompt-response) pairs into explicit instructions for the LLM. Prepend an instruction to the start of the dialog with Summarize the following conversation and to the start of the summary with Summary as follows:

Training prompt (dialogue):

Summarize the following conversation.

    Chris: This is his part of the conversation.
    Antje: This is her part of the conversation.
    
Summary: 
Training response (summary):

Both Chris and Antje participated in the conversation.
Then preprocess the prompt-response dataset into tokens and pull out their input_ids (1 per token).

In [29]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

In [30]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1500
    })
})

In [31]:
small_tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

In [32]:
small_tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
})

fully fine-tuned version of the model we call it **instruct model **

In [33]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained('nash5657/flan-t5-instruct', torch_dtype=torch.bfloat16)

In [34]:
small_model = AutoModelForSeq2SeqLM.from_pretrained('nash5657/t5-small-dataset', torch_dtype=torch.bfloat16)

set up the PEFT/LoRA model for fine-tuning with a new layer/parameter adapter. Using PEFT/LoRA, you are freezing the underlying LLM and only training the adapter. Have a look at the LoRA configuration below. Note the rank (r) hyper-parameter, which defines the rank/dimension of the adapter to be trained.

In [35]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [36]:
peft_model = get_peft_model(original_model, 
                            lora_config)
print(peft_model.print_trainable_parameters())

trainable params: 3538944 || all params: 251116800 || trainable%: 1.4092820552029972
None


In [37]:
#peft_model

In [38]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-1, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    #logging_steps=1e-1,
    #max_steps=10   
)
    
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=small_tokenized_datasets["train"],
)

In [ ]:
peft_trainer.train()

Step,Training Loss


Prepare this model by adding an adapter to the original FLAN-T5 model. You are setting `is_trainable=False` because the plan is only to perform inference with this PEFT model. If you were preparing the model for further training, you would set `is_trainable=True`.

In [45]:
peft_trainer.model.save_pretrained('./test/peft_trainer_v/')

In [32]:
#peft_trainer.merge_and_unload()

In [101]:
from peft import PeftModel, PeftConfig
peft_model = PeftModel.from_pretrained(original_model, 
                                       './test/peft_trainer_v/', 
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

In [94]:
#peft_model.merge_and_unload()

In [102]:
print(peft_model.print_trainable_parameters())

trainable params: 0 || all params: 251116800 || trainable%: 0.0
None


In [16]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

In [17]:

original_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
        prompt = f"""
    Summarize the following conversation.

    {dialogue}

    Summary: """
        
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids

        original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
        original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
        
        original_model_summaries.append(original_model_text_output)

        
        peft_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
        peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

        peft_model_summaries.append(peft_model_text_output)

    
    # Evaluate
rouge = evaluate.load('rouge')
    
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
        predictions=peft_model_summaries,
        references=human_baseline_summaries[0:len(peft_model_summaries)],
        use_aggregator=True,
        use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.23781826118922889, 'rouge2': 0.11402387041773232, 'rougeL': 0.21746611065159452, 'rougeLsum': 0.21637010614833194}
PEFT MODEL:
{'rouge1': 0.3837300511435397, 'rouge2': 0.15814273845528468, 'rougeL': 0.2864151527401888, 'rougeLsum': 0.28696973739266163}


In [18]:
print("Absolute percentage improvement of INSTRUCT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of INSTRUCT MODEL over ORIGINAL MODEL
rouge1: 14.59%
rouge2: 4.41%
rougeL: 6.89%
rougeLsum: 7.06%


In [19]:
original_model_summaries

['#Person1#: I need to take a dictation for you.',
 '#Person1#: I need to take a dictation for you.',
 '#Person1#: I need to take a dictation for you.',
 'The traffic jam at the Carrefour intersection is a problem.',
 'The traffic jam at the Carrefour intersection is a problem.',
 'The traffic jam at the Carrefour intersection is a problem.',
 'Masha and Hero are getting divorced.',
 'Masha and Hero are getting divorced.',
 'Masha and Hero are getting divorced.',
 "#Person1#: Happy birthday, Brian. #Person2#: I'm so happy you're having a good time. #Person1#: Thank you, I'm sure you're having a good time. #Person2#: Thank you, I'm sure you're having a good time. #Person1#: Thank you, I'm sure you're having a good time. #Person2#: Thank you, I'm sure you're having a good time. #Person1#: Thank you, I'm sure you're having a good time."]

In [20]:
peft_model_summaries

['#Person1# asks Ms. Dawson to take a dictation to all employees by this afternoon. Ms. Dawson tells #Person1# all office communications are restricted to email correspondence and official memos. #Person1# wants to change the communication methods and Ms. Dawson says it applies to internal and external communications.',
 '#Person1# asks Ms. Dawson to take a dictation to all employees by this afternoon. Ms. Dawson tells #Person1# all office communications are restricted to email correspondence and official memos. #Person1# wants to change the communication methods and Ms. Dawson says it applies to internal and external communications.',
 '#Person1# asks Ms. Dawson to take a dictation to all employees by this afternoon. Ms. Dawson tells #Person1# all office communications are restricted to email correspondence and official memos. #Person1# wants to change the communication methods and Ms. Dawson says it applies to internal and external communications.',
 "#Person2# got stuck in traffic a